In [12]:
import numpy as np
import requests

class DayThree:
    def __init__(self, target):
        self.target = target
        self.solution = 0
        self.printOutput = False
        self.run_problem()
        self.print_solution()
        
    def run_problem(self):
        pass
    
    def print_solution(self):
        pass

In [18]:
class PartOne(DayThree):
    def __init__(self, target):
        super().__init__(target)       
        
    def print_solution(self):
        print('Minimum number of steps is: {}'.format(int(self.solution)))
        
    def find_number_in_level(self, first_number_in_level, level_width):
        current_coord = (level_width, 2 if level_width>2 else 1)
        corners = [(level_width, level_width), (1, level_width), (1,1), (level_width, 1)]
        n_steps = self.target - first_number_in_level
        current_directon_id = 0
        def step(current_coord, current_directon_id):
            current_directon_id = current_directon_id%4
            x_loc = current_coord[0]
            y_loc = current_coord[1]
            if current_directon_id == 0:
                y_loc += 1
            elif current_directon_id == 2:
                y_loc -= 1
            if current_directon_id == 1:
                x_loc -= 1
            elif current_directon_id == 3:
                x_loc += 1
            next_loc = (x_loc, y_loc)
            if next_loc in corners:
                current_directon_id += 1
            return next_loc, current_directon_id
        for i in range(n_steps):
            current_coord, current_directon_id = step(current_coord, current_directon_id)
        return current_coord
    
    def find_level_of_number(self):
        current_width = 1
        cumulative_sum = 0
        level = 0

        while cumulative_sum < self.target:
            level += 1
            numbers_in_level = 2*current_width + 2*(current_width-2) if level is not 1 else 1
            cumulative_sum += numbers_in_level
            first_number_in_level = cumulative_sum - numbers_in_level + 1
            return_width = current_width
            current_width = current_width + 2
        return return_width, first_number_in_level
    
    def distance_to_middle(self, location, width):
        half_size = (width + 1)/2 if width > 1 else 1
        midpoint = (half_size, half_size)
        x_moves = abs(location[0] - midpoint[0])
        y_moves = abs(location[1] - midpoint[1])
        return x_moves + y_moves
    
    def run_problem(self):
        width, first_number = self.find_level_of_number()
        location = self.find_number_in_level(first_number, width)
        self.solution = self.distance_to_middle(location, width)

In [10]:
class PartTwo(DayThree):
    def __init__(self, target):
        self.reached_target = False
        super().__init__(target)
        
    def run_problem(self):
        self.current_width = 1
        self.grid = np.ones((self.current_width, self.current_width))
        self.position = (1,1)
        self.fill_levels()
        
    def fill_levels(self):
        while not self.reached_target:
            self.upgrade()
            self.fill()        
            
    def print_solution(self):
        print('Largest written value was: {}'.format(int(self.solution)))
        
    def fill(self):
        while self.is_not_filled():
            self.fill_value_at_position()
            self.step()
        self.print_grid()
            
    def print_grid(self):
        if self.printOutput:
            print(np.flipud(self.grid.T))
            
    def fill_value_at_position(self):
        x_choices = [-1, 0, 1]
        y_choices = [-1, 0, 1]
        running_count = 0
        for x in x_choices:
            for y in y_choices:
                actual_x = self.position[0] + x
                actual_y = self.position[1] + y
                if actual_y < 0 or actual_x < 0 or actual_y >= self.current_width or actual_x >= self.current_width:
                    continue
                running_count += self.grid[actual_x, actual_y]
        if running_count > self.target and not self.reached_target:
            self.solution = running_count
            self.reached_target = True
        self.grid[self.position] = running_count
        
    def is_not_filled(self):
        return np.any(self.grid==0)
    
    def step(self):
        self.current_directon_id = self.current_directon_id%4
        x_loc = self.position[0]
        y_loc = self.position[1]
        if self.current_directon_id == 0:
            y_loc += 1
        elif self.current_directon_id == 2:
            y_loc -= 1
        if self.current_directon_id == 1:
            x_loc -= 1
        elif self.current_directon_id == 3:
            x_loc += 1
        next_loc = (x_loc, y_loc)
        if next_loc in self.corners:
            self.current_directon_id += 1
        self.position = next_loc
        
    def upgrade(self):
        self.current_width += 2
        old_grid = self.grid
        self.grid = np.zeros((self.current_width, self.current_width))
        self.grid[1:-1, 1:-1] = old_grid
        self.corners = [(self.current_width-1, self.current_width-1), (0, self.current_width-1), (0, 0), (self.current_width-1, 0)]
        self.position = (self.current_width-1, 1 if self.current_width > 2 else 0)
        self.current_directon_id = 0

In [26]:
target = int(requests.get('https://aoc.intae.it/3').text)
distance_finder = PartOne(target)
value_writer = PartTwo(target)

Minimum number of steps is: 326
Largest written value was: 363010
